In [31]:
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import os
from pathlib import Path
home = str(Path.home())

In [40]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)  
    clf = LogisticRegression(n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [41]:
def predictions(clf, X_test, scaler=None):
    if not scaler == None:
        X_test   = scaler.transform(X_test)
    y_hat    = clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    return y_hat, y_hat_pr

In [42]:
def RF(X_train, y_train, X_test, y_test):
    kwargs = {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 60, 'bootstrap': False}  

    clf = RandomForestClassifier(n_jobs=-1, **kwargs)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [43]:
factor = 0.8
nr_mean = 3
mean_tables = []
nr_samples=2000

attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
detectors = ['lid', 'multilid']

results = {}

for it in range(nr_mean):
    random_state = [21, 30, 65][it] # random.randint(100)
    print("random state: ", random_state)
    final_table = np.zeros((len(attacks), 2))
    base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
    
    results[it] = {}
    
    for dataset in datasets:
        
        if dataset == 'imagenet':
            k=30
        else:
            k=20
        
        for model in models:
            
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
                
            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}
                
            #print("config: ", random_state, dataset, model)
            
            for detector in detectors:
                
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]

                for rows in range(len(nor)):
                    X_nor = nor[rows]
                    X_adv = adv[rows]
                    if len(X_nor.shape) > 2: 
                        X_nor = nor[rows].reshape((nor[rows].shape[0], -1))
                        X_adv = adv[rows].reshape((adv[rows].shape[0], -1))

                    y_nor = np.zeros(X_nor.shape[0]).astype('int')
                    y_adv = np.ones(X_nor.shape[0]).astype('int')

                    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(X_nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
                    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(X_adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

                    X_train = np.concatenate((x_train_n, x_train_a))
                    y_train = np.concatenate((y_train_n, y_train_a))

                    X_test = np.concatenate((x_test_n, x_test_a))
                    y_test = np.concatenate((y_test_n, y_test_a))

                    if detector == 'lid':
                        y_hat, y_hat_pr = LR(X_train, y_train, X_test, y_test)
                    elif detector == 'multilid':
                        y_hat, y_hat_pr = RF(X_train, y_train, X_test, y_test)

                    auc = round(100*roc_auc_score(y_test, y_hat_pr), 2)
                    f1 =  round(100*f1_score(y_test, y_hat), 2)
     
                    if not detector in results[it][dataset][model]:
                        results[it][dataset][model][detector] = {}
                    
                    if not attacks[rows] in results[it][dataset][model][detector]: 
                        results[it][dataset][model][detector][attacks[rows]] = {}
                    
                    
                    if not 'auc' in results[it][dataset][model][detector][attacks[rows]]:
                        results[it][dataset][model][detector][attacks[rows]]['auc'] = {}
                    if not 'f1' in results[it][dataset][model][detector][attacks[rows]]:
                        results[it][dataset][model][detector][attacks[rows]]['f1' ] = {}
                        
                    results[it][dataset][model][detector][attacks[rows]]['auc'] = auc
                    results[it][dataset][model][detector][attacks[rows]]['f1'] = f1                    

                    # print(attacks[rows].upper(), '&' , auc, '&', f1)

random state:  21
random state:  30
random state:  65


# LID

In [44]:
lid_list_mean = []
for it_mean in range(nr_mean):
    lid_list = []
    for dataset in ['cifar10', 'cifar100', 'imagenet']:
        for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            tmp = np.zeros((len(attacks), 2))
            for it, val in enumerate(results[it_mean][dataset][model]['lid'].items()):
                #print(it, val)
                tmp[it, 0] = val[1]['auc'].copy()
                tmp[it, 1] = val[1]['f1'].copy()
            lid_list.append(tmp.copy())
    
    lid_list_mean.append(np.stack(lid_list))
lid_list_mean_res = np.stack(lid_list_mean)
lid_mean = np.mean(lid_list_mean_res, axis=0)
lid_var = np.var(lid_list_mean_res, axis=0)

In [45]:
latex_table = ""
for j in range(lid_mean.shape[1]):
    latex_table += "\\textbf{"+f"{attacks[j].upper()}"+"} & "
    for i in range(lid_mean.shape[0]):
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
    latex_table = latex_table[:-2] + "\\\\\n"

# Print the LaTeX table
print(latex_table)

\textbf{FGSM} & $96.07 \pm 0.07$ & $89.58 \pm 0.16$ & $88.11 \pm 0.25$ & $79.36 \pm 0.34$ & $98.21 \pm 0.48$ & $93.61 \pm 1.57$ & $77.98 \pm 1.19$ & $70.75 \pm 0.58$ & $71.44 \pm 8.46$ & $66.22 \pm 2.31$ \\
\textbf{BIM} & $87.09 \pm 0.21$ & $79.70 \pm 0.40$ & $92.47 \pm 1.69$ & $84.55 \pm 1.31$ & $95.61 \pm 0.90$ & $88.90 \pm 0.74$ & $81.62 \pm 3.42$ & $73.95 \pm 5.96$ & $94.46 \pm 0.28$ & $87.20 \pm 1.25$ \\
\textbf{PGD} & $89.28 \pm 0.62$ & $80.37 \pm 2.37$ & $89.80 \pm 0.39$ & $80.49 \pm 1.79$ & $97.83 \pm 0.06$ & $91.85 \pm 0.12$ & $85.29 \pm 1.20$ & $78.86 \pm 1.92$ & $96.21 \pm 0.43$ & $89.76 \pm 2.03$ \\
\textbf{AA} & $96.86 \pm 0.41$ & $91.13 \pm 0.68$ & $97.06 \pm 0.13$ & $91.03 \pm 0.18$ & $99.32 \pm 0.02$ & $95.61 \pm 0.57$ & $87.27 \pm 0.94$ & $79.05 \pm 0.04$ & $99.97 \pm 0.00$ & $99.45 \pm 0.11$ \\
\textbf{DF} & $95.37 \pm 0.04$ & $87.71 \pm 2.39$ & $86.46 \pm 0.26$ & $77.00 \pm 1.03$ & $57.04 \pm 0.62$ & $53.01 \pm 0.71$ & $54.41 \pm 0.03$ & $53.12 \pm 0.55$ & $54.65 \pm

# multiLID

In [38]:
lid_list_mean = []
for it_mean in range(nr_mean):
    lid_list = []
    for dataset in ['cifar10', 'cifar100', 'imagenet']:
        for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            tmp = np.zeros((len(attacks), 2))
            for it, val in enumerate(results[it_mean][dataset][model]['multilid'].items()):
                #print(it, val)
                tmp[it, 0] = val[1]['auc'].copy()
                tmp[it, 1] = val[1]['f1'].copy()
            lid_list.append(tmp.copy())
    
    lid_list_mean.append(np.stack(lid_list))
lid_list_mean_res = np.stack(lid_list_mean)
lid_mean = np.mean(lid_list_mean_res, axis=0)
lid_var = np.var(lid_list_mean_res, axis=0)

In [39]:
latex_table = ""
for j in range(lid_mean.shape[1]):
    latex_table += "\\textbf{"+f"{attacks[j].upper()}"+"} & "
    for i in range(lid_mean.shape[0]):
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
    latex_table = latex_table[:-2] + "\\\\\n"

# Print the LaTeX table
print(latex_table)

\textbf{FGSM} & $96.98 \pm 0.20$ & $91.19 \pm 0.95$ & $90.78 \pm 0.38$ & $82.34 \pm 1.37$ & $98.55 \pm 0.28$ & $94.80 \pm 1.32$ & $83.00 \pm 0.84$ & $76.89 \pm 0.02$ & $79.25 \pm 2.75$ & $72.98 \pm 0.67$ \\
\textbf{BIM} & $96.10 \pm 0.39$ & $89.93 \pm 1.24$ & $94.50 \pm 0.26$ & $88.14 \pm 0.43$ & $97.88 \pm 0.07$ & $91.72 \pm 0.09$ & $82.96 \pm 0.94$ & $75.42 \pm 1.39$ & $94.48 \pm 0.18$ & $86.92 \pm 0.93$ \\
\textbf{PGD} & $97.69 \pm 0.12$ & $92.81 \pm 0.37$ & $92.35 \pm 1.46$ & $83.52 \pm 3.46$ & $98.76 \pm 0.05$ & $94.74 \pm 1.14$ & $88.39 \pm 0.06$ & $81.42 \pm 0.44$ & $96.39 \pm 0.11$ & $90.21 \pm 0.49$ \\
\textbf{AA} & $99.45 \pm 0.03$ & $96.88 \pm 0.00$ & $98.77 \pm 0.08$ & $94.85 \pm 1.42$ & $99.85 \pm 0.00$ & $98.33 \pm 0.01$ & $91.25 \pm 0.33$ & $83.48 \pm 0.17$ & $99.90 \pm 0.00$ & $98.83 \pm 0.04$ \\
\textbf{DF} & $97.51 \pm 0.12$ & $94.04 \pm 0.26$ & $89.37 \pm 2.42$ & $84.32 \pm 0.83$ & $74.75 \pm 0.16$ & $70.18 \pm 0.57$ & $73.78 \pm 1.12$ & $70.04 \pm 0.09$ & $52.93 \pm